In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math
from embfts.util.DataSetUtil import DataSetUtil


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
data_set_util = DataSetUtil()

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/air/air_quality_beijing.csv', sep=',')
df = df.drop(labels=['No','day','year','month','hour','wd','station'], axis=1)
df = df.drop(labels=['No.1','day.1','year.1','month.1','hour.1','wd.1','station.1'], axis=1)
df = df.drop(labels=['No.2','day.2','year.2','month.2','hour.2','wd.2','station.2'], axis=1)
df = df.drop(labels=['No.3','day.3','year.3','month.3','hour.3','wd.3','station.3'], axis=1)
df = df.drop(labels=['No.4','day.4','year.4','month.4','hour.4','wd.4','station.4'], axis=1)
df = df.drop(labels=['No.5','day.5','year.5','month.5','hour.5','wd.5','station.5'], axis=1)
df = df.drop(labels=['No.6','day.6','year.6','month.6','hour.6','wd.6','station.6'], axis=1)
df = df.drop(labels=['No.7','day.7','year.7','month.7','hour.7','wd.7','station.7'], axis=1)
df = df.drop(labels=['No.8','day.8','year.8','month.8','hour.8','wd.8','station.8'], axis=1)
df = df.drop(labels=['No.9','day.9','year.9','month.9','hour.9','wd.9','station.9'], axis=1)
df = df.drop(labels=['No.10','day.10','year.10','month.10','hour.10','wd.10','station.10'], axis=1)
df = df.drop(labels=['No.11','day.11','year.11','month.11','hour.11','wd.11','station.11'], axis=1)
df.dropna(inplace=True)
data = data_set_util.clean_dataset(df)
data.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,PM10.11,SO2.11,NO2.11,CO.11,O3.11,TEMP.11,PRES.11,DEWP.11,RAIN.11,WSPM.11
23,24.0,24.0,26.0,54.0,600.0,36.0,-0.2,1030.5,-17.4,0.0,...,15.0,14.0,41.0,500.0,31.0,1.0,1029.6,-18.4,0.0,1.8
24,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,...,10.0,11.0,35.0,400.0,39.0,0.5,1030.2,-18.3,0.0,1.4
25,14.0,17.0,21.0,36.0,400.0,50.0,-1.0,1031.3,-17.3,0.0,...,8.0,9.0,33.0,400.0,40.0,-0.3,1030.2,-17.5,0.0,1.0
32,3.0,7.0,21.0,49.0,500.0,43.0,-0.4,1029.6,-17.6,0.0,...,7.0,17.0,53.0,500.0,17.0,0.0,1028.8,-16.8,0.0,1.3
33,10.0,14.0,47.0,62.0,700.0,29.0,0.6,1029.7,-16.7,0.0,...,11.0,17.0,50.0,500.0,23.0,1.1,1028.9,-16.8,0.0,1.0


In [5]:
data.describe()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,PM10.11,SO2.11,NO2.11,CO.11,O3.11,TEMP.11,PRES.11,DEWP.11,RAIN.11,WSPM.11
count,17117.000000,17117.000000,17117.000000,17117.000000,17117.000000,17117.000000,17117.000000,1.711700e+04,17117.000000,17117.000000,...,17117.000000,17117.000000,17117.000000,17117.000000,17117.000000,17117.000000,1.711700e+04,17117.000000,17117.000000,17117.000000
mean,83.296273,110.694170,15.999007,60.439440,1333.241865,63.306736,9.459692,1.072132e+03,1.863364,0.056780,...,112.914763,15.840153,56.153567,1378.364725,59.617327,9.631463,1.071910e+03,1.547029,0.060221,1.757043
std,82.384679,94.281669,20.084888,38.192672,1302.951080,740.002232,706.739740,7.728333e+03,13.731930,0.681897,...,95.566311,20.650638,35.882851,1227.163859,640.476953,706.740278,7.728334e+03,13.666349,0.722386,1.194589
min,3.000000,3.000000,1.000000,2.000000,100.000000,1.000000,-9575.000000,9.878000e+02,-35.300000,0.000000,...,3.000000,1.000000,2.000000,100.000000,1.000000,-9575.000000,9.878000e+02,-35.300000,0.000000,0.000000
25%,21.000000,37.000000,3.000000,31.000000,500.000000,8.000000,2.400000,1.004500e+03,-9.200000,0.000000,...,40.000000,2.000000,29.000000,600.000000,8.000000,2.600000,1.004100e+03,-9.400000,0.000000,0.900000
50%,58.000000,86.000000,8.000000,54.000000,900.000000,43.000000,12.600000,1.012500e+03,1.400000,0.000000,...,90.000000,8.000000,49.000000,1000.000000,42.000000,12.800000,1.012100e+03,1.000000,0.000000,1.400000
75%,116.000000,158.000000,20.000000,84.000000,1600.000000,80.000000,22.700000,1.021500e+03,14.500000,0.000000,...,155.000000,21.000000,77.000000,1700.000000,79.000000,22.900000,1.021600e+03,14.200000,0.000000,2.300000
max,713.000000,948.000000,185.000000,290.000000,10000.000000,78183.000000,21475.000000,1.012125e+06,28.500000,29.300000,...,811.000000,272.000000,235.000000,9800.000000,83538.000000,21475.000000,1.012125e+06,28.000000,29.300000,13.200000


In [6]:
# frame as supervised learning
reframed = data_set_util.series_to_supervised_mimo(data, 1, 1)
reframed.head()

,PM2.5(t-1),PM10(t-1),SO2(t-1),NO2(t-1),CO(t-1),O3(t-1),TEMP(t-1),PRES(t-1),DEWP(t-1),RAIN(t-1),...,PM10.11(t),SO2.11(t),NO2.11(t),CO.11(t),O3.11(t),TEMP.11(t),PRES.11(t),DEWP.11(t),RAIN.11(t),WSPM.11(t)
24,24.0,24.0,26.0,54.0,600.0,36.0,-0.2,1030.5,-17.4,0.0,...,10.0,11.0,35.0,400.0,39.0,0.5,1030.2,-18.3,0.0,1.4
25,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,...,8.0,9.0,33.0,400.0,40.0,-0.3,1030.2,-17.5,0.0,1.0
32,14.0,17.0,21.0,36.0,400.0,50.0,-1.0,1031.3,-17.3,0.0,...,7.0,17.0,53.0,500.0,17.0,0.0,1028.8,-16.8,0.0,1.3
33,3.0,7.0,21.0,49.0,500.0,43.0,-0.4,1029.6,-17.6,0.0,...,11.0,17.0,50.0,500.0,23.0,1.1,1028.9,-16.8,0.0,1.0
34,10.0,14.0,47.0,62.0,700.0,29.0,0.6,1029.7,-16.7,0.0,...,14.0,22.0,52.0,600.0,20.0,1.9,1028.4,-17.1,0.0,2.0


In [7]:
df_forecats_columns = data.loc[:,'PM2.5':'WSPM.11'].columns

In [8]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,'PM2.5(t-1)':'WSPM.11(t-1)']
            df_test = ttest.loc[:,'PM2.5(t-1)':'WSPM.11(t-1)']
            df_original = ttest.loc[:,'PM2.5(t)':'WSPM.11(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            df_original = pd.DataFrame(df_original.values,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

## Sliding Window: MIMO KPCA-NSFTS

In [9]:
num_components_pca = 10
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [10]:
kpca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 570
--------------------
training window 1140
--------------------
training window 1710
--------------------
training window 2280
--------------------
training window 2850
--------------------
training window 3420
--------------------
training window 3990
--------------------
training window 4560
--------------------
training window 5130


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 5700


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 6270
--------------------
training window 6840
--------------------
training window 7410
--------------------
training window 7980
--------------------
training window 8550
--------------------
training window 9120
--------------------
training window 9690
--------------------
training window 10260
--------------------
training window 10830
--------------------
training window 11400
--------------------
training window 11970


...

--------------------
training window 12540
--------------------
training window 13110
--------------------
training window 13680
--------------------
training window 14250
--------------------
training window 14820
--------------------
training window 15390


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 15960


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 16530


In [11]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

kpca_result.to_csv (r'all_windows_kpca_mo_ensfts_uci_air_quality_beijing_all.csv', index = False, header=True)

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 


In [12]:
print("Statistics MIMO KPCA-NSFTS(test): ")
pd.set_option('display.max_rows', None)
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5,22.203,12.709,34.213,0.820,13.383,0.094,11.294,4.867,14.322,0.156,4.702,0.020
1,PM10,32.419,19.829,32.191,0.697,13.479,0.114,12.441,6.262,8.627,0.217,3.249,0.027
2,SO2,5.317,3.130,27.455,0.762,11.555,0.098,4.284,2.605,14.310,0.119,4.702,0.019
3,NO2,14.393,9.489,26.623,0.705,10.582,0.121,3.487,2.050,13.322,0.167,2.704,0.025
4,CO,390.209,227.274,25.524,0.727,10.280,0.112,289.788,148.043,19.098,0.173,3.893,0.034
5,O3,399.994,76.986,132.678,0.673,21.204,0.131,2087.778,352.812,333.723,0.342,10.516,0.031
6,TEMP,228.671,108.308,inf,0.727,17.922,0.108,886.647,407.742,NaN,0.362,21.180,0.042
7,PRES,4001.809,484.921,24.499,0.850,0.622,0.065,21912.497,2651.628,133.753,0.365,3.191,0.028
8,DEWP,1.605,0.979,inf,0.868,8.579,0.086,0.605,0.302,NaN,0.086,7.784,0.018
9,RAIN,0.283,0.064,NaN,-0.075,NaN,NaN,0.676,0.169,NaN,0.485,NaN,NaN


In [13]:
final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_air_quality_beijing_all_with_std.csv', index = False, header=True)

In [14]:
#final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_air_quality_beijing_all_2.csv', index = False, header=True)

## Sliding Window: MIMO PCA-NSFTS

In [15]:
num_components_pca = 10
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [16]:
pca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 570
--------------------
training window 1140


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 1710


...

--------------------
training window 12540
--------------------
training window 13110
--------------------
training window 13680
--------------------
training window 14250
--------------------
training window 14820
--------------------
training window 15390


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 15960


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 16530


In [17]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

pca_result.to_csv (r'all_windows_pca_mo_ensfts_uci_air_quality_beijing_all.csv', index = False, header=True)

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 
           

print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5,23.923,16.327,52.963,0.806,18.848,0.101,11.102,7.307,24.029,0.108,6.975,0.019
1,PM10,33.904,23.701,46.808,0.696,17.253,0.119,13.375,8.987,17.295,0.158,5.469,0.025
2,SO2,6.849,4.743,53.863,0.653,19.932,0.120,7.044,4.680,23.787,0.146,5.675,0.024
3,NO2,16.141,11.882,34.716,0.645,13.248,0.134,5.512,4.008,14.386,0.200,3.626,0.031
4,CO,428.157,299.602,36.806,0.691,14.617,0.122,293.021,207.614,24.320,0.154,6.635,0.030
5,O3,279.834,96.633,567.986,0.671,28.223,0.139,1421.484,448.057,2358.072,0.176,16.899,0.031
6,TEMP,247.688,162.085,inf,0.615,21.059,0.132,949.614,621.145,NaN,0.408,23.375,0.045
7,PRES,4344.115,1672.987,141.158,0.730,3.201,0.103,23783.139,9155.605,772.395,0.421,17.150,0.037
8,DEWP,1.954,1.391,inf,0.819,11.490,0.103,0.784,0.536,NaN,0.088,10.207,0.019
9,RAIN,0.275,0.136,inf,-0.080,98.567,NaN,0.652,0.357,NaN,0.407,1.729,NaN


In [18]:
pd.set_option('display.max_rows', None)
final_measures_pca

,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5,23.923,16.327,52.963,0.806,18.848,0.101,11.102,7.307,24.029,0.108,6.975,0.019
1,PM10,33.904,23.701,46.808,0.696,17.253,0.119,13.375,8.987,17.295,0.158,5.469,0.025
2,SO2,6.849,4.743,53.863,0.653,19.932,0.120,7.044,4.680,23.787,0.146,5.675,0.024
3,NO2,16.141,11.882,34.716,0.645,13.248,0.134,5.512,4.008,14.386,0.200,3.626,0.031
4,CO,428.157,299.602,36.806,0.691,14.617,0.122,293.021,207.614,24.320,0.154,6.635,0.030
5,O3,279.834,96.633,567.986,0.671,28.223,0.139,1421.484,448.057,2358.072,0.176,16.899,0.031
6,TEMP,247.688,162.085,inf,0.615,21.059,0.132,949.614,621.145,NaN,0.408,23.375,0.045
7,PRES,4344.115,1672.987,141.158,0.730,3.201,0.103,23783.139,9155.605,772.395,0.421,17.150,0.037
8,DEWP,1.954,1.391,inf,0.819,11.490,0.103,0.784,0.536,NaN,0.088,10.207,0.019
9,RAIN,0.275,0.136,inf,-0.080,98.567,NaN,0.652,0.357,NaN,0.407,1.729,NaN


In [19]:
#final_measures_pca.to_csv (r'pca_mo_ensfts_uci_air_quality_beijing_all.csv', index = False, header=True)

In [20]:
final_measures_pca.to_csv (r'pca_mo_ensfts_uci_air_quality_beijing_all_with_std.csv', index = False, header=True)

In [21]:
pd.set_option('display.max_rows', None)
final_measures_pca

,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5,23.923,16.327,52.963,0.806,18.848,0.101,11.102,7.307,24.029,0.108,6.975,0.019
1,PM10,33.904,23.701,46.808,0.696,17.253,0.119,13.375,8.987,17.295,0.158,5.469,0.025
2,SO2,6.849,4.743,53.863,0.653,19.932,0.120,7.044,4.680,23.787,0.146,5.675,0.024
3,NO2,16.141,11.882,34.716,0.645,13.248,0.134,5.512,4.008,14.386,0.200,3.626,0.031
4,CO,428.157,299.602,36.806,0.691,14.617,0.122,293.021,207.614,24.320,0.154,6.635,0.030
5,O3,279.834,96.633,567.986,0.671,28.223,0.139,1421.484,448.057,2358.072,0.176,16.899,0.031
6,TEMP,247.688,162.085,inf,0.615,21.059,0.132,949.614,621.145,NaN,0.408,23.375,0.045
7,PRES,4344.115,1672.987,141.158,0.730,3.201,0.103,23783.139,9155.605,772.395,0.421,17.150,0.037
8,DEWP,1.954,1.391,inf,0.819,11.490,0.103,0.784,0.536,NaN,0.088,10.207,0.019
9,RAIN,0.275,0.136,inf,-0.080,98.567,NaN,0.652,0.357,NaN,0.407,1.729,NaN
